# Frame Difference
As outlined in motion detection research paper. Frame difference compares 2 frames to capture the dynamic information between frames. The steps is summarized below:
1. Collect video inputs -> then cut of sequence frames (converting it to frames).
2. Binarization: Algorithm creates pixel values representing motion detected in each difference frame.
3. Output of Binarization: Black and white picture frame where white represents motion detected.

Note: Will change it such that black encapsulates motion detected instead.

## Import Dependencies

In [1]:
!pip install opencv-python numpy fastdtw tensorflow scikit-learn matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time

## Implmenting Frame Difference Algorithm

In [3]:
def frame_diff(prev_frame, current_frame, next_frame):
    if prev_frame is None or current_frame is None or next_frame is None:
        return None
    
    #standarized frame.
    prev_frame = cv2.resize(prev_frame, (640, 480))
    current_frame = cv2.resize(current_frame, (640, 480))
    next_frame = cv2.resize(next_frame, (640, 480))
    
    #getting the absolute difference between current frame and next frame.
    diff1 = cv2.absdiff(next_frame, current_frame)
    
    #absolute diff between previous and current frame.
    diff2 = cv2.absdiff(current_frame, prev_frame)
    
    #bitwise AND operation to obtain common region of motion
    motion_diff = cv2.bitwise_and(diff1, diff2)
    
    return motion_diff

In [3]:
#Capture video from input video
#''
cap = cv2.VideoCapture('vid_test_frame/sorry.mp4')

#read first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

while True:
    #frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    #grayscale it so black/white output only.
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    #threshold of motion difference.
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    #display frame difference.
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
    
    #read the next frames.
    prev_frame = current_frame
    current_frame = next_frame
    if not ret:
        #don't motion diff when there is no frames.
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()
    
    #Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#release video capture and close windows.
cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


#### Frame Difference Live Code
Allow frame differencing method to implement in real-time gesture recognition method. Also pixel reduction is applied here

In [46]:
# Capture video from the default camera
cap = cv2.VideoCapture(0)

# Set lower resolution, for example, 320x240
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

# Read the first three frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

while True:
    # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    if motion_diff is not None:
        # Convert to grayscale for thresholding
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
        
        # Display the result
        cv2.imshow("Motion Diff Frame", thresh)

    # Update frames
    prev_frame = current_frame
    current_frame = next_frame
    ret, next_frame = cap.read()

    if not ret:
        print("Failed to capture frame. Exiting ...")
        break

    # Break loop on 'q' keypress
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

### Frame difference with extracted matrix.

In [6]:
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_01.mp4')
#D:\BSL_project\bsl_dataset\manual-script\sorry

#read first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

# Counter for the number of frames extracted
frame_count = 0

# List to store extracted frames
extracted_frames = []

while True:
    # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    # Grayscale it so black/white output only
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    # Threshold of motion difference
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    # Display frame difference
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
        
        # Append the thresholded frame to the list
        thresh_copy = thresh.copy()
        motion_diff_flatten = thresh_copy.flatten()
        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
        extracted_frames.append(motion_diff_flatten)
        frame_count += 1

    # Read the next frames
    prev_frame = current_frame
    current_frame = next_frame

    if not ret:
        # Don't motion diff when there are no frames
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()

    # Break the loop if 'q' is pressed or 15 frames are extracted
    if cv2.waitKey(1) & 0xFF == ord('q') or frame_count == 70:
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()



In [8]:
# Convert the list of frames to a NumPy array
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)

Shape of the extracted frames array: (70, 307200)


In [9]:
np.sum(extracted_frames_array)

49326180

#### Applying Normalization of Extracted Matrices
To ensure consistent array across different sign speeds.
There are several approach we can take:
1. Fixed Length Segment
2. Key frame extraction
3. Time Wrapping
4. Speed/Velocity over time
5. Temporal Pooling
6. Zero padding - for very short signs.

In [41]:
# build up from this
# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_01.mp4')

# Read the first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

# Counter for the number of frames extracted
frame_count = 0

# Length of each segment
segment_length = 70

# List to store extracted frames
extracted_frames = []

while True:
    # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    # Grayscale it so black/white output only
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    # Threshold of motion difference
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    
    # Display frame difference
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
        thresh_copy = thresh.copy()
        # Append the thresholded frame to the list
        #motion_diff_flatten = thresh_copy.flatten()
        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
        extracted_frames.append(thresh_copy)
        frame_count += 1

    # Read the next frames
    prev_frame = current_frame
    current_frame = next_frame

    if not ret:
        # Don't motion diff when there are no frames
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()

    # Break the loop if 'q' is pressed or the specified segment length is reached
    if cv2.waitKey(1) & 0xFF == ord('q') or frame_count % segment_length == 0:
        # Reset frame count for the next segment
        frame_count = 0
        break
        # Optional: Do something with the frames in the extracted_frames list

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...


In [43]:
# Convert the list of frames to a NumPy array
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)
extracted_frames_array

Shape of the extracted frames array: (70, 480, 640)


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [20]:
np.sum(extracted_frames_array)

49326180

In [13]:

from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [4,4]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)

2.8284271247461903


##### Normalization of Matrices
For short videos (fast sign speed) -> zero pad the rows of zeros, aligning them as our standard matrix size of 100 rows.

For long videos (slow sign speed):
1. Linear interpolation: 2 known value points interpolate them.
2. Resize with averaging: resized matrix may not preserve the original pattern/dynamics of data.

In [14]:
# Varying video length varies the sequence length.
# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_01.mp4')

# Read the first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

# Counter for the number of frames extracted
frame_count = 0

# Length of each segment
segment_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# List to store extracted frames
extracted_frames = []

while True:
    # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    # Grayscale it so black/white output only
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    # Threshold of motion difference
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    
    # Display frame difference
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
        thresh_copy = thresh.copy()
        # Append the thresholded frame to the list
        motion_diff_flatten = thresh_copy.flatten()
        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
        extracted_frames.append(motion_diff_flatten)
        frame_count += 1

    # Read the next frames
    prev_frame = current_frame
    current_frame = next_frame

    if not ret:
        # Don't motion diff when there are no frames
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()

    # Break the loop if 'q' is pressed or the specified segment length is reached
    if cv2.waitKey(1) & 0xFF == ord('q') or frame_count % segment_length == 0:
        # Reset frame count for the next segment
        frame_count = 0
        break
        # Optional: Do something with the frames in the extracted_frames list

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...


In [21]:
segment_length

70

In [22]:
# Convert the list of frames to a NumPy array
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)
array_shape = extracted_frames_array.shape
# Calculate the number of columns needed for 100 rows
num_columns_needed = array_shape[1] * array_shape[0] // 100

# Specify the new shape for reshaping (100 rows, calculated number of columns)
new_shape = (100, 307200)


reshape_extracted_frame = np.resize(extracted_frames_array,new_shape)
print ("Reshaped to:", reshape_extracted_frame.shape)

Shape of the extracted frames array: (70, 307200)
Reshaped to: (100, 307200)


In [23]:
print ("sum" , np.sum(extracted_frames_array))
print ("resized sum: ",np.sum(reshape_extracted_frame))

sum 49326180
resized sum:  68538900


#### Live Matrix Extraction with Normalization
Implement matrix extraction of the frame in real-time and apply normalization

In [51]:
# Varying video length varies the sequence length.
# Capture video from input video
cap = cv2.VideoCapture(0)

# Set lower resolution, for example, 320x240
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

# Read the first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

# Counter for the number of frames extracted
frame_count = 0

# Length of each segment
segment_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# List to store extracted frames
extracted_frames = []

while True:
    # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    # Grayscale it so black/white output only
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    # Threshold of motion difference
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    
    # Display frame difference
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
        thresh_copy = thresh.copy()
        # Append the thresholded frame to the list
        motion_diff_flatten = thresh_copy.flatten()
        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
        extracted_frames.append(motion_diff_flatten)
        frame_count += 1

    # Read the next frames
    prev_frame = current_frame
    current_frame = next_frame

    if not ret:
        # Don't motion diff when there are no frames
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()

    # Break the loop if 'q' is pressed or the specified segment length is reached
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Reset frame count for the next segment
        frame_count = 0
        break
        # Optional: Do something with the frames in the extracted_frames list

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

In [52]:
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)

Shape of the extracted frames array: (110, 307200)


### Zero-padding and Resize Together

Zero padding functionality and resize after the video is processed

In [94]:
#cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_02.mp4')
cap = cv2.VideoCapture('bsl_dataset/bsl-dictionary-dataset/own_dataset/sorry/sorry_test1.mp4')
#long video: D:\BSL_project\bsl_dataset\bsl-dictionary-dataset\own_dataset\sorry

# Determine the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Read the first 3 frames
ret, prev_frame = cap.read()
ret, current_frame = cap.read()
ret, next_frame = cap.read()

# Counter for the number of frames processed
frame_count = 0

# Length of each segment
segment_length = 70

# List to store extracted frames
extracted_frames = []


while True:
     # Frame differencing technique
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    # Grayscale it so black/white output only
    if motion_diff is not None:
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
    
    # Threshold of motion difference
    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
    
    # Display frame difference
    if thresh is not None:
        # Display the frames
        cv2.imshow("Motion Diff Frame", thresh)
        thresh_copy = thresh.copy()
        # Append the thresholded frame to the list
        motion_diff_flatten = thresh_copy.flatten()
        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
        extracted_frames.append(motion_diff_flatten)
        frame_count += 1

    # Read the next frames
    prev_frame = current_frame
    current_frame = next_frame

    if not ret:
        # Don't motion diff when there are no frames
        print("Can't receive frame (stream end?). Exiting ...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, next_frame = cap.read()
        continue
    else:
        ret, next_frame = cap.read()

    # Break the loop if 'q' is pressed or the specified segment length is reached
    if cv2.waitKey(1) & 0xFF == ord('q') or frame_count % segment_length == 0:
        
        # Apply zero-padding if video is shorter than the segment length
        if len(extracted_frames) < segment_length:
        # Calculate how many frames need to be padded
            padding_count = segment_length - len(extracted_frames)
            # Get the size of the flattened frame for padding
            frame_size = extracted_frames[0].size
            # Append zero frames for padding
            for _ in range(padding_count):
                zero_padding = np.zeros(frame_size, dtype=np.uint8)
                extracted_frames.append(zero_padding)
        elif len(extracted_frames) > segment_length:
            # If the list length is more than segment_length, resize it
            extracted_frames = np.resize(extracted_frames, (segment_length, extracted_frames[0].size))
        # Reset frame count for the next segment
        frame_count = 0
        break
        # Optional: Do something with the frames in the extracted_frames list


# After processing, you may have a list that exceeds the segment length if you didn't break the loop
# or simply use it as is for further processing

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

In [5]:
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)

Shape of the extracted frames array: (70, 307200)


In [25]:
extracted_frames

[array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)]

# Redo Implementation - Unflatten Test

In [102]:
# Capture video from input video.
cap = cv2.VideoCapture('bsl_dataset/bsl-dictionary-dataset/own_dataset/sorry/sorry_test1.mp4')
#cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_02.mp4')

# Initialize list to store the reshaped frames.
extracted_frames = []


while True:
    # Read the first 3 frames.
    ret, prev_frame = cap.read()
    ret, current_frame = cap.read()
    ret, next_frame = cap.read()
    if not ret:
        break  # End of video or failed read.
    
    # Apply frame differencing technique.
    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
    
    if motion_diff is not None:
        #convert to grayscale
        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
        cv2.imshow("Motion Diff Frame", thresh)
        # Instead of flattening, keep the frames in their original 2D shape.
        extracted_frames.append(thresh)
    
    # Prepare for the next iteration.
    prev_frame = current_frame
    current_frame = next_frame
    
# Stack frames along a new dimension for CNN input.
if extracted_frames:
    stacked_frames = np.stack(extracted_frames, axis=0)
    # Add a channel dimension (assuming the CNN expects 'channels_first' format).
    stacked_frames = stacked_frames[:, np.newaxis, :, :]
    # Now stacked_frames is in the shape: (num_frames, 1, height, width).

cap.release()
cv2.destroyAllWindows()

In [103]:
extracted_frames_array = np.array(extracted_frames)
print("Shape of the extracted frames array:", extracted_frames_array.shape)
stacked_frames_array = np.array(stacked_frames)
print("Shape of the extracted frames array:", stacked_frames_array.shape)

Shape of the extracted frames array: (31, 480, 640)
Shape of the extracted frames array: (31, 1, 480, 640)


# Motion History Image Method - CNN Shape
Motion History Image (MHI) represent motion in sequence frames as a single image. The intensity of each pixel in the MHI corresponds to the recency of motion at that location - brighter the pixel, more recent the location.

Algorithm generates MHI that captures the temporal aspects of motion by decaying older movements and highlight new ones - using further analysis like action recognition on 2DCNN.

In [147]:
def update_mhi(prev_frame, current_frame, mhi, decay=0.7):
    """
    Update the motion history image based on the current frame and the previous frame.
    """
    # Compute the absolute difference between the current and the previous frame
    frame_diff = cv2.absdiff(current_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, binary_diff = cv2.threshold(gray_diff, 30, 255, cv2.THRESH_BINARY)

    # Convert binary_diff to float32 to match mhi's type
    binary_diff = np.float32(binary_diff / 255)  # Normalize to [0, 1] to maintain consistency

    # Update MHI: decay existing MHI values and increase values where motion is detected
    mhi = cv2.add(mhi * decay, binary_diff)
    return mhi


In [4]:
# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_03.mp4')
#cap = cv2.VideoCapture('bsl_dataset/bsl-dictionary-dataset/own_dataset/sorry/sorry_test1.mp4')

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Initialize MHI
height, width = prev_frame.shape[:2]
mhi = np.zeros((height, width), dtype=np.float32)

# List to store MHIs
mhis = []

# Process video
while True:
    ret, current_frame = cap.read()
    if not ret:
        break  # End of video
    
    # Update MHI
    mhi = update_mhi(prev_frame, current_frame, mhi)
    
    # Prepare for next iteration
    prev_frame = current_frame.copy()

    # Optionally, visualize the MHI
    normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
    cv2.imshow("MHI", normalized_mhi)

    # Store the MHI for this frame
    mhis.append(mhi.copy())

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
# Save the MHI image to a file
output_path = 'mhi_image2.png'  # Specify the output file path
cv2.imwrite(output_path, normalized_mhi)
cap.release()
cv2.destroyAllWindows()
# Resize and normalize stacked_mhis as needed for your CNN input requirements


# Stack MHIs for CNN input
# Here we assume the CNN is designed to take inputs of shape (height, width, channels)
# where channels correspond to different time steps in the MHI sequence
# Ensure the length of mhis matches your CNN's expected depth or adjust as necessary
stacked_mhis = np.stack(mhis, axis=-1)
stacked_mhis = np.expand_dims(stacked_mhis, axis=0)  # Add batch dimension

ValueError: operands could not be broadcast together with shapes (720,1280) (720,1280,3) 

In [190]:
# Function to update MHI, adapted to handle color (BGR) frames directly
def update_mhi(prev_frame, current_frame, mhi, decay=0.02):
    frame_diff = cv2.absdiff(current_frame, prev_frame)
    # Applying threshold to get binary motion detection, works on grayscale
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, binary_diff = cv2.threshold(gray_diff, 30, 255, cv2.THRESH_BINARY)
    binary_diff = np.repeat(binary_diff[:, :, np.newaxis], 3, axis=2)  # Make it 3-channel
    binary_diff = np.float32(binary_diff / 255)  # Normalize to [0, 1]

    # Update the MHI
    mhi = cv2.add(mhi * decay, binary_diff)
    return mhi

In [4]:
def update_mhi(prev_frame, current_frame, mhi, decay=0.09):
    """
    Update the motion history image (MHI) to retain motion across the entire video,
    adapted for a 3-channel (color) MHI.
    
    - prev_frame: The previous frame in the video (BGR).
    - current_frame: The current frame in the video (BGR).
    - mhi: The current state of the MHI, a 3-channel image.
    - decay: The rate at which previous motion history fades.
    """
    # Compute the absolute difference between the current and the previous frame
    frame_diff = cv2.absdiff(current_frame, prev_frame)
    
    # Convert the frame difference to grayscale and threshold it
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, motion_mask = cv2.threshold(gray_diff, 25, 1, cv2.THRESH_BINARY)
    
    # Convert the single-channel motion mask to a 3-channel mask
    motion_mask_3ch = cv2.merge([motion_mask, motion_mask, motion_mask])
    
    # Update the MHI: New motion is added with a value of 1, and existing motion history decays
    # Ensure that both mhi and motion_mask_3ch are floats for correct operation
    mhi = (mhi * (1 - decay)) + np.float32(motion_mask_3ch)
    
    # Ensure MHI values are capped at 1
    mhi = np.clip(mhi, 0, 1)
    
    return mhi

In [17]:


# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_28.mp4')

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Initialize MHI with 3 channels to keep color information
mhi = np.zeros((*prev_frame.shape[:2], 3), dtype=np.float32)

# List to store MHIs
mhis = []

# Process video to generate MHI
while True:
    ret, current_frame = cap.read()
    if not ret:
        break  # End of video
    
    # Update MHI
    mhi = update_mhi(prev_frame, current_frame, mhi)
    
    # Prepare for next iteration
    prev_frame = current_frame.copy()

    # Optionally, visualize the MHI
    normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
    cv2.imshow("MHI", normalized_mhi)

    # Store the MHI for this frame
    mhis.append(mhi.copy())

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the last MHI image to a file
output_path = 'mhi_image28_alt_0.09.png'  # Adjust the output path as needed
cv2.imwrite(output_path, normalized_mhi)

# Clean up
cap.release()
cv2.destroyAllWindows()

# NOTE: Further processing to fit MHIs into MobileNetV2 would follow here,
# including resizing and normalization as needed for your application.

In [5]:
#USE THIS
# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_28.mp4')

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Initialize MHI with 3 channels to keep color information
mhi = np.zeros((*prev_frame.shape[:2], 3), dtype=np.float32)

# List to store MHIs
mhis = []

# Process video to generate MHI
while True:
    ret, current_frame = cap.read()
    if not ret:
        break  # End of video
    
    # Update MHI
    mhi = update_mhi(prev_frame, current_frame, mhi)
    
    # Prepare for next iteration
    prev_frame = current_frame.copy()

    # Optionally, visualize the MHI
    normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
    cv2.imshow("MHI", normalized_mhi)
    
    resized_mhi = cv2.resize(normalized_mhi, (200, 200))

    # Store the MHI for this frame
    mhis.append(resized_mhi.copy())

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the last MHI image to a file
output_path = 'mhi_image28_alt_0.09.png'  # Adjust the output path as needed
cv2.imwrite(output_path, resized_mhi)

# Clean up
cap.release()
cv2.destroyAllWindows()

# NOTE: Further processing to fit MHIs into MobileNetV2 would follow here,
# including resizing and normalization as needed for your application.

In [6]:
output_path
mhi_array = np.array(mhi)
normalized_mhi_array = np.array(resized_mhi)
print("Shape of the mhi_array:", mhi_array.shape)
#print("Shape of stacked_mhi:", stacked_mhis.shape)
print("Shape of normalized:", resized_mhi.shape)

Shape of the mhi_array: (720, 1280, 3)
Shape of normalized: (200, 200, 3)


## Motion History Image - 3 channel decay method - (throw away)
Idea: Instead of duplicating the MHI into 3 channel for RBG into mobile net, set different decay rate and log it into the shape.
Then the input should be fed into MobileNetv2.

In [7]:
#changed approach, each mhi is defined using this method then combined at the end with a wrapper function
def update_mhi_single_chan(prev_frame, current_frame, mhi, decay):
    # Compute the absolute difference between the current and the previous frame
    frame_diff = cv2.absdiff(current_frame, prev_frame)
    
    # Convert the frame difference to grayscale and threshold it
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, motion_mask = cv2.threshold(gray_diff, 25, 1, cv2.THRESH_BINARY)
    
    # Convert the single-channel motion mask to a 3-channel mask
    #motion_mask_3ch = cv2.merge([motion_mask, motion_mask, motion_mask])
    
    # Update the MHI: New motion is added with a value of 1, and existing motion history decays
    # Ensure that both mhi and motion_mask_3ch are floats for correct operation
    mhi = (mhi * (1 - decay)) + np.float32(motion_mask)
    
    # Ensure MHI values are capped at 1
    mhi = np.clip(mhi, 0, 1)
    
    return mhi

def mhi_3channel(prev_frame, current_frame, mhi_log):
    mhi_1, mhi_2, mhi_3 = cv2.split(mhi_log) 
    mhi_1 = update_mhi_single_chan(prev_frame, current_frame, mhi_1, 0.0009)
    mhi_2 = update_mhi_single_chan(prev_frame, current_frame, mhi_2, 0.009)
    mhi_3 = update_mhi_single_chan(prev_frame, current_frame, mhi_3, 0.9)
    
    mhi_3ch = cv2.merge([mhi_1, mhi_2, mhi_3])
    mhi_3ch_final = np.clip(mhi_3ch, 0, 1)
    
    return mhi_3ch_final
    

In [8]:
#USE THIS
# Capture video from input video
cap = cv2.VideoCapture('bsl_dataset/manual-script/sorry/sorry_28.mp4')

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Initialize MHI with 3 channels to keep color information
mhi = np.zeros((*prev_frame.shape[:2], 3), dtype=np.float32)

# List to store MHIs
mhis = []

# Process video to generate MHI
while True:
    ret, current_frame = cap.read()
    if not ret:
        break  # End of video
    
    # Update MHI
    mhi = mhi_3channel(prev_frame, current_frame, mhi)
    
    # Prepare for next iteration
    prev_frame = current_frame.copy()

    # Optionally, visualize the MHI
    normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
    cv2.imshow("MHI_3channel", normalized_mhi)
    
    resized_mhi = cv2.resize(normalized_mhi, (200, 200))

    # Store the MHI for this frame
    mhis.append(resized_mhi.copy())

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the last MHI image to a file
output_path = 'mhi_image_3decay.png'  # Adjust the output path as needed
cv2.imwrite(output_path, resized_mhi)
#cv2.imshow(resized_mhi)

# Clean up
cap.release()
cv2.destroyAllWindows()

# NOTE: Further processing to fit MHIs into MobileNetV2 would follow here,
# including resizing and normalization as needed for your application.

In [9]:
#check the output shape of the combined mhi 
mhi_array = np.array(mhi)
normalized_mhi_array = np.array(resized_mhi)
print("Shape of the mhi_array:", mhi_array.shape)
#print("Shape of stacked_mhi:", stacked_mhis.shape)
print("Shape of normalized:", resized_mhi.shape)

#optional plot the different image - side by side comparison.


Shape of the mhi_array: (720, 1280, 3)
Shape of normalized: (200, 200, 3)


# Extracting Image to Matrix
After performing frame differencing and background subtraction. The next step is to sum the pixels into a matrix format.
Steps:
- Sum all horizontal axis into an array in a column down format.
- sum all vertical axis into array in a row left to right format.
- Put horizontal axis array into matrix where the rows are the time. filling up the array one by one.
- apply transformation of the vertical array so it becomes horizontal alignment.
- Repeat the horizontal axis method -> add up arrays that gets input with time.

The matrix will be fed into the ML model.

Q: How to verify that the mathematical operation is correct?
- perform operations manually on small example with known values.go through it manually.
- use known input (e.g. sorry video)
- unit testing -> create unit test for each maths operation/function. Write test cases. (implement this after checking calculation is right)

cut up frames?

Problem: How to deal with this kind of processing in real time?

View the above implementation


In [8]:
extracted_frames_array.shape

NameError: name 'extracted_frames_array' is not defined

In [9]:
np.save('test_fd_matrix.npy', extracted_frames_array)

In [10]:
np.load('test_fd_matrix.npy')

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

# Setup Folders for Collection

In [7]:
#Defining Data
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MHI_Data') 

# Actions that we try to detect
actions = np.array(['thanks', 'sorry'])

# Thirty videos worth of data
no_sequences = 100


In [17]:
#Creation of folders
## create folders for each sequence in each signs.
#30 videos so 30 folders within each sign
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Extracted Matrix Values for Training

Loop through the videos in the video folder, they are labelled with ids that separate them.
Then for each subfolder get the actions and loop through it, collecting their frame difference matrices.

### Data augmentation from code
Applying matrix transformation through the numpy arrays - thus data augmentation.

In [58]:
#Data augmentation Step
class CONFIG:

    ROOT_DIRECTORY = "asl_dataset_word/archive (1)"
    #os.path.join("..","data", "WLASL")""
    #JSON_FILE = "WLASL_v0.3.json"
    #NSLT_FILE = "nslt_100.json"
    VIDEO_FOLDER = "extracted_videos_asl"
    #mean=[0.485, 0.456, 0.406]
    #std=[0.229, 0.224, 0.225]
    mean = [0, 0, 0]
    std = [1,1,1]
    DEBUG = True
    
    P_OF_TRANSFORM = 0.9
    P_OF_TRANSFORM_COLOR = 0.2
    
    SHIFT_LIMIT=0.1
    SCALE_LIMIT=0.1
    ROTATE_LIMIT=10
    
    # set to small, when prototyping, or 0 when deploying to cloud or PC with loads of RAM
    DATA_LIMIT = 100
    FRAME_SIZE = 30
    
    
    BATCH_SIZE = 8
    PORTION_OF_DATA_FOR_TRAINING = 0.8
    

In [56]:
import os 
#import torch
import random
import numpy as np
from tqdm import tqdm
from typing import List
import cv2

#import torch
#import torch.functional as F
#from torch.utils.data import Dataset, DataLoader
#import torchvision.transforms as transforms
        
class SignRecognitionDataset(keras.utils.Sequence):
    def __init__(self, max_start : int, max_end) -> None:
        # setup the paths
        video_path = os.path.join(CONFIG.ROOT_DIRECTORY, CONFIG.VIDEO_FOLDER)
        dataset_description = os.path.join(CONFIG.ROOT_DIRECTORY)

        # load the filepaths for videos
        self.video_paths = [os.path.join(video_path, file) for file in os.listdir(video_path)]

        
        self.videos_paths = []
        self.paths_not_found = []
        self.labels = []
        self.start_frames = []
        self.end_frames = []
        data_items = []    
        

        #for el in tqdm(dataset_json.items()):
        for action_folder in os.listdir(VIDEO_FOLDER):
            action_path = os.path.join(ROOT_DIRECTORy, action_folder)
            
            if os.path.isdir(action_path) and (actions is None or action in actions):
                print(f"Processing action: {action}")
            for video_file in tqdm(os.listdir(action_path)):
                if video_file.endswith(".mp4"):
                    video_id, properties = el[0], el[1]
                    path = os.path.join(video_path, video_id + ".mp4")
                    if not os.path.exists(path):
                        self.paths_not_found.append(path)
                        continue
            subset = properties["subset"]
            label, start, end = properties["action"]
            
            if start > max_start:
                continue
                
            if end > max_end:
                continue
            
            self.videos_paths.append(path)
            self.labels.append(label)
            self.start_frames.append(start)
            self.end_frames.append(end)
    
        self.videos_paths = np.array(self.video_paths)
        self.paths_not_found = np.array(self.paths_not_found)
        self.labels = np.array(self.labels)
        self.start_frames = np.array(self.start_frames)
        self.end_frames = np.array(self.end_frames)

        self.unique_labels = np.unique(self.labels)
        
    
    def preprocess_trajectory(self, traj : List[np.ndarray]):
        return traj
    
    def __len__(self):
        return len(self.videos_paths)

    def __getitem__(self, idx):
        path, label = self.videos_paths[idx], self.labels[idx]
        trajectory = SignRecognitionDataset.get_video(path)
        
        return self.preprocess_trajectory(trajectory), label
        
        
    @staticmethod
    def get_video(video_path : str) -> List[np.ndarray]:

        if not os.path.exists(video_path):
            return None

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return None

        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                frame_array = np.array(frame)
                frames.append(cv2.cvtColor(frame_array, cv2.COLOR_BGR2RGB))
            else:
                break

        cap.release()
        return np.array(frames)

    @staticmethod
    def rescale_video(frames : np.ndarray, desired_shape) -> np.ndarray:
        refined = []
        for img in frames: 
            y, x, c = img.shape
            cropped = img[:, (x // 2 - y//2) : (x // 2 + y//2), :]
            scaled = cv2.resize(cropped, desired_shape)
            refined.append(scaled)
    
        return np.array(refined)
        
    
ds = SignRecognitionDataset(max_start=1, max_end=150)
print(f"after filtering : size of dataset={len(ds)}")

NameError: name 'dataset_json' is not defined

In [59]:

class SignRecognitionDataset(keras.utils.Sequence):
    def __init__(self, max_start: int, max_end: int):
        self.video_path = os.path.join(CONFIG.ROOT_DIRECTORY, CONFIG.VIDEO_FOLDER)
        self.video_paths = [os.path.join(self.video_path, file) for file in os.listdir(self.video_path) if file.endswith('.mp4')]
        
        # Apply limits
        self.video_paths = self.video_paths[:CONFIG.DATA_LIMIT]  # Limiting the number of videos processed

        # Set up image augmentation (with TensorFlow)
        self.augment = keras.Sequential([
            keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            keras.layers.experimental.preprocessing.RandomRotation(CONFIG.ROTATE_LIMIT / 360),
            keras.layers.experimental.preprocessing.RandomZoom(CONFIG.SCALE_LIMIT),
            keras.layers.experimental.preprocessing.RandomTranslation(CONFIG.SHIFT_LIMIT, CONFIG.SHIFT_LIMIT)
        ])

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        frames = self.get_video_frames(video_path)
        augmented_frames = self.augment(frames)  # Apply augmentation
        return augmented_frames

    def get_video_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
            else:
                break
        cap.release()
        frames = np.array(frames)
        if len(frames) > 0:
            frames = tf.image.resize(frames, (CONFIG.FRAME_SIZE, CONFIG.FRAME_SIZE))
        return frames

# Example of using the dataset
ds = SignRecognitionDataset(max_start=1, max_end=150)
print(f"After filtering: size of dataset = {len(ds)}")

After filtering: size of dataset = 0


In [6]:
#Folder of dataset
dataset_folder = "bsl_dataset/manual-script"
#D:\BSL_project\bsl_dataset\manual-script

In [69]:
#Loop through all subfolders in the dataset folder
for action_folder in os.listdir(dataset_folder):
    action_path = os.path.join(dataset_folder, action_folder)
    if os.path.isdir(action_path):
        print(f"Processing action: {action_folder}")
        
        #Initialize sequence counter for each video
        sequence_counter = 0

        
        #Loop through all video files in the current action subfolder
        for video_file in os.listdir(action_path):
            if video_file.endswith(".mp4"):
                video_path = os.path.join(action_path, video_file)
                print(f"Processing video: {video_file}")
                #Extract the action (label) from subfolder name
                action = action_folder
                
                #Create folders for each sequence in sign (if no folders)
                for sequence in range (no_sequences):
                    try:
                        os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
                    except:
                        #folder, exist just pass
                        pass
                #display current video on frame
                cap = cv2.VideoCapture(video_path)
                # Determine the total number of frames in the video
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                
                #Loop through videos
                frame_num = 0
                
                #read first 3 frames
                ret, prev_frame = cap.read()
                ret, current_frame = cap.read()
                ret, next_frame = cap.read()

                # Counter for the number of frames processed
                frame_count = 0
                
                # Length of each segment
                segment_length = 12

                # List to store extracted frames
                extracted_frames = []

                
                while cap.isOpened():
                    ret, frame = cap.read()
                    
                    #Image processing part
                    #frame differencing technique
                    motion_diff = frame_diff(prev_frame, current_frame, next_frame)
                        
                    # Grayscale it so black/white output only
                    if motion_diff is not None:
                        motion_diff_grayscale = cv2.cvtColor(motion_diff, cv2.COLOR_BGR2GRAY)
                    #threshold of motion difference.
                    _, thresh = cv2.threshold(motion_diff_grayscale, 30, 255, cv2.THRESH_BINARY)
    
                    # Display frame difference
                    if thresh is not None:
                        # Display the frames
                        #cv2.imshow("Motion Diff Frame", thresh)
                        thresh_copy = thresh.copy()
                        # Append the thresholded frame to the list
                        motion_diff_flatten = thresh_copy.flatten()
                        # flatten to 1 Dimensional array. Now as array goes down wrt to time.
                        extracted_frames.append(motion_diff_flatten)
                        frame_count += 1
                    
                    cv2.imshow('FD Feed', thresh)
                    
                    
                    
                    #read the next frames.
                    prev_frame = current_frame
                    current_frame = next_frame
                    if not ret:
                        break
                    else:
                        ret, next_frame = cap.read()
                    
                    # Export Matrix Logic - Normalized and Fitted.
                     # Break gracefully
                    if cv2.waitKey(1) & 0xFF == ord('q') or frame_count % segment_length == 0:
                        # Apply zero-padding if video is shorter than the segment length
                        if len(extracted_frames) < segment_length:
                            # Calculate how many frames need to be padded
                            padding_count = segment_length - len(extracted_frames)
                            # Get the size of the flattened frame for padding
                            frame_size = extracted_frames[0].size
                            # Append zero frames for padding
                            for _ in range(padding_count):
                                zero_padding = np.zeros(frame_size, dtype=np.uint8)
                                extracted_frames.append(zero_padding)
                        elif len(extracted_frames) > segment_length:
                            # If the list length is more than segment_length, resize it
                            extracted_frames = np.resize(extracted_frames, (segment_length, extracted_frames[0].size))
                            # Reset frame count for the next segment
                        npy_path = os.path.join(DATA_PATH, action_folder, str(sequence_counter), f"{segment_length}.npy")
                        np.save(npy_path, extracted_frames)
                        print(f"Saved FD matrix to: {npy_path}")
                        frame_count = 0
                        break

                frame_num += 1
                cap.release()
                cv2.destroyAllWindows()  # Close the window after processing each video

                # Increment sequence counter for the next video
                sequence_counter += 1
                    

Processing action: thanks
Processing video: thanks_38.mp4
Saved FD matrix to: FD_Data\thanks\0\12.npy
Processing video: thanks_01.mp4
Saved FD matrix to: FD_Data\thanks\1\12.npy
Processing video: thanks_02.mp4
Saved FD matrix to: FD_Data\thanks\2\12.npy
Processing video: thanks_03.mp4
Processing video: thanks_04.mp4
Saved FD matrix to: FD_Data\thanks\4\12.npy
Processing video: thanks_05.mp4
Saved FD matrix to: FD_Data\thanks\5\12.npy
Processing video: thanks_06.mp4
Saved FD matrix to: FD_Data\thanks\6\12.npy
Processing video: thanks_07.mp4
Saved FD matrix to: FD_Data\thanks\7\12.npy
Processing video: thanks_08.mp4
Saved FD matrix to: FD_Data\thanks\8\12.npy
Processing video: thanks_09.mp4
Saved FD matrix to: FD_Data\thanks\9\12.npy
Processing video: thanks_10.mp4
Saved FD matrix to: FD_Data\thanks\10\12.npy
Processing video: thanks_11.mp4
Saved FD matrix to: FD_Data\thanks\11\12.npy
Processing video: thanks_12.mp4
Saved FD matrix to: FD_Data\thanks\12\12.npy
Processing video: thanks_13

In [34]:
#data from BSL code version


from tqdm import tqdm
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

for action_folder in os.listdir(dataset_folder):
    action_path = os.path.join(dataset_folder, action_folder)
    if os.path.isdir(action_path) and (actions is None or action in actions):
        print(f"Processing action: {action}")
        for video_file in tqdm(os.listdir(action_path)):
            if video_file.endswith(".mp4"):
                video_path = os.path.join(action_path, video_file)
                cap = cv2.VideoCapture(video_path)

                ret, prev_frame = cap.read()
                if not ret:
                    print("Failed to read video", video_path)
                    cap.release()
                    continue

                mhi = np.zeros((*prev_frame.shape[:2], 3), dtype=np.float32)
                mhis = []

                while True:
                    ret, current_frame = cap.read()
                    if not ret:
                        break

                    mhi = update_mhi(prev_frame, current_frame, mhi)
                    prev_frame = current_frame.copy()
                    
                    normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
                    

                    #Resize MHI here - for MobileNetV2
                    resized_mhi = cv2.resize(normalized_mhi, (200, 200))
                    
                    #store MHI for this frame
                    mhis.append(resized_mhi.copy())

                

                # Generate an output path for the MHI npy file
                output_npy_path = os.path.join(DATA_PATH, action_folder,  f"{video_file.split('.')[0]}.npy")
                # Save the MHI list as a .npy file
                np.save(output_npy_path, np.array(resized_mhi))
                #output_path = os.path.join(DATA_PATH, action_folder,  f"{video_file.split('.')[0]}.png")
                #cv2.imwrite(output_path, resized_mhi)

cap.release()
cv2.destroyAllWindows()  # Make sure to destroy all cv2 windows outside the loop

Processing action: thanks


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\mered\AppData\Local\Temp\ipykernel_19684\379117340.py:31: RuntimeWarning: invalid value encountered in divide
  normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
C:\Users\mered\AppData\Local\Temp\ipykernel_19684\379117340.py:31: RuntimeWarning: invalid value encountered in cast
  normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:03<00:00,  1.24s/it]


Processing action: thanks


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:57<00:00,  1.18s/it]


test the shape of the npy file saved fits MobileNetv2 expected shape of (200, 200, 3) shape.

In [9]:
npy_file_path = 'MHI_Data/thanks/thanks_02.npy'  # Adjust the path as needed

# Load the npy file
test_array = np.load(npy_file_path)

# Output the shape of the array
print(test_array.shape)

(200, 200, 3)


In [8]:
DATA_PATH = os.path.join('MHI_ASL_DATA') 

In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# dataset from asl:
dataset_folder = "asl_dataset_word/archive (1)/extracted_videos_asl"  # Updated to point to your dataset directory

# Defining Data
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MHI_ASL_DATA') 

# Actions will be dynamically populated based on folder names
actions = []

# Check and create DATA_PATH if not exists
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Populate actions based on folder names in dataset_folder
for entry in os.listdir(dataset_folder):
    if os.path.isdir(os.path.join(dataset_folder, entry)):
        actions.append(entry)

actions = np.array(actions)  # Convert list of actions to numpy array

print(f"Detected actions: {actions}")

for action in actions:
    action_folder = os.path.join(dataset_folder, action)
    action_data_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_data_path):
        os.makedirs(action_data_path)
    
    print(f"Processing action: {action}")
    for video_file in tqdm(os.listdir(action_folder)):
        if video_file.endswith(".mp4"):
            # Generate an output path for the MHI npy file
            output_npy_path = os.path.join(action_data_path, f"{video_file.split('.')[0]}.npy")
            
            

            video_path = os.path.join(action_folder, video_file)
            cap = cv2.VideoCapture(video_path)

            ret, prev_frame = cap.read()
            if not ret:
                print("Failed to read video", video_path)
                cap.release()
                continue

            mhi = np.zeros((*prev_frame.shape[:2], 3), dtype=np.float32)
            mhis = []

            while True:
                ret, current_frame = cap.read()
                if not ret:
                    break

                mhi = update_mhi(prev_frame, current_frame, mhi)
                prev_frame = current_frame.copy()
                #normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
                normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
                resized_mhi = cv2.resize(normalized_mhi, (200, 200))
                mhis.append(resized_mhi.copy())

            # Save the MHI list as a .npy file
            np.save(output_npy_path, np.array(resized_mhi.copy()))
            #output_path_img = os.path.join(action_data_path, f"{video_file.split('.')[0]}.png")
            #cv2.imwrite(output_path_img, resized_mhi)
            cap.release()

            

cv2.destroyAllWindows()  # Make sure to destroy all cv2 windows


Detected actions: ['book' 'drink' 'computer' ... 'weigh' 'wheelchair' 'whistle']
Processing action: book


 67%|████████████████████████████████████████████████████████                            | 4/6 [00:07<00:02,  1.49s/it]C:\Users\mered\AppData\Local\Temp\ipykernel_30156\1027205552.py:63: RuntimeWarning: invalid value encountered in divide
  normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
C:\Users\mered\AppData\Local\Temp\ipykernel_30156\1027205552.py:63: RuntimeWarning: invalid value encountered in cast
  normalized_mhi = np.uint8(np.clip((mhi / mhi.max()) * 255, 0, 255))
100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.46s/it]


Processing action: drink


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:21<00:00,  1.41s/it]


Processing action: computer


...

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.78it/s]


Processing action: wheelchair


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.09it/s]


Processing action: whistle


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.83it/s]


In [30]:
#code that loops through folders to get actions
import os
import numpy as np

# Path to the dataset folder containing subfolders for each action
dataset_folder = "asl_dataset_word/archive (1)/extracted_videos_asl"  # Update this to your dataset directory

# Path for exported data (not used for video processing here, but kept for structure)
DATA_PATH = os.path.join('MHI_ASL_DATA') 

# Ensure DATA_PATH exists
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# List to hold the names of actions based on folder names in the dataset_folder
actions = []

# Populate actions list with the names of directories in dataset_folder
for entry in os.listdir(dataset_folder):
    if os.path.isdir(os.path.join(dataset_folder, entry)):
        actions.append(entry)

# Convert the list of actions to a numpy array (optional, depending on further use)
actions = np.array(actions)

# Print detected actions
print(f"Detected actions: {actions}")

Detected actions: ['book' 'drink' 'computer' ... 'weigh' 'wheelchair' 'whistle']


In [40]:
import os

M = 0
labl = None
root = "MHI_ASL_DATA"
for  el in os.listdir(root):
    path = os.path.join(root, el)
    #print (f"{el} -> {len(os.listdir(path))}")
    v = len(os.listdir(path))
    if M < v:
        M =v 
        labl = el
    
print(labl, M) - 

before 16


In [10]:
#test the size of the npy file has the same image size frame
npy_file_path = 'MHI_ASL_Data/drink/17709.npy'  # Adjust the path as needed

# Load the npy file
test_array = np.load(npy_file_path)

# Output the shape of the array
print(test_array.shape)

#list out all the action labels we have
print(actions.shape)

(200, 200, 3)
(2000,)


# Preprocess Data, Create Labels and Features

In [11]:
#train_test is a function to split dataset into training and testing set
from sklearn.model_selection import train_test_split

#Converts class vectors to binary class matrix for categorial crossentropy
from tensorflow.keras.utils import to_categorical

In [31]:
#Create Label Map
label_map = {label:num for num, label in enumerate(actions)}

In [32]:
label_map

{'book': 0,
 'drink': 1,
 'computer': 2,
 'before': 3,
 'chair': 4,
 'go': 5,
 'clothes': 6,
 'who': 7,
 'candy': 8,
 'cousin': 9,
 'deaf': 10,
 'fine': 11,
 'help': 12,
 'no': 13,
 'thin': 14,
 'walk': 15,
 'year': 16,
 'yes': 17,
 'all': 18,
 'black': 19,
 'cool': 20,
 'finish': 21,
 'hot': 22,
 'like': 23,
 'many': 24,
 'mother': 25,
 'now': 26,
 'orange': 27,
 'table': 28,
 'thanksgiving': 29,
 'what': 30,
 'woman': 31,
 'bed': 32,
 'blue': 33,
 'bowling': 34,
 'can': 35,
 'dog': 36,
 'family': 37,
 'fish': 38,
 'graduate': 39,
 'hat': 40,
 'hearing': 41,
 'kiss': 42,
 'language': 43,
 'later': 44,
 'man': 45,
 'shirt': 46,
 'study': 47,
 'tall': 48,
 'white': 49,
 'wrong': 50,
 'accident': 51,
 'apple': 52,
 'bird': 53,
 'change': 54,
 'color': 55,
 'corn': 56,
 'cow': 57,
 'dance': 58,
 'dark': 59,
 'doctor': 60,
 'eat': 61,
 'enjoy': 62,
 'forget': 63,
 'give': 64,
 'last': 65,
 'meet': 66,
 'pink': 67,
 'pizza': 68,
 'play': 69,
 'school': 70,
 'secretary': 71,
 'short': 72,
 '

In [81]:
#res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))

#sequences = contains the full array of the matrix sequence
# labels = contains the actions associated with the matrix.
# Initialize lists to hold your sequences (X) and labels (Y)
sequences, labels = [], []
# Loop through each action
for action in actions:
    # Loop through each sequence number for the current action
    for sequence in range(1, no_sequences - 1):  # assuming sequence folders start from 1 to 50
        # Load the .npy file for the current sequence
        # The file is specifically named '10.npy' as per your dataset structure
        file_path = os.path.join(DATA_PATH, action, str(sequence), "12.npy")
        extracted_frames = np.load(file_path)
        
        # Append the extracted frames to the sequences list
        sequences.append(extracted_frames)
        
        # Append the corresponding label to the labels list
        labels.append(label_map[action])

# At this point, 'sequences' holds your X (the data) and 'labels' holds your Y (the labels)
# Convert sequences and labels to numpy arrays if needed for your machine learning framework



In [14]:
#For MHI - use this.
sequences, labels = [], []
for action_folder in os.listdir(DATA_PATH):
    action_path = os.path.join(DATA_PATH, action_folder)
    if os.path.isdir(action_path):
        for file_name in os.listdir(action_path):
            if file_name.endswith('.npy'):
                # Load MHI data
                mhi_path = os.path.join(action_path, file_name)
                mhi_data = np.load(mhi_path)
                
                sequences.append(mhi_data)
                
                # Use the label_map to convert action names to integers
                labels.append(label_map[action_folder])

In [15]:
np.array(sequences).shape

(11950, 200, 200, 3)

In [16]:
np.array(labels).shape

(11950,)

In [17]:
X = np.array(sequences)
X.shape

(11950, 200, 200, 3)

In [18]:
#from sklearn.preprocessing import LabelEncoder
#label_encoder = LabelEncoder()
#integer_encoded = label_encoder.fit_transform(labels)
y = to_categorical(labels).astype(int)
#y = to_categorical(integer_encoded)

In [19]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [20]:
#Here, split into train, test, and dev (validate) dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

# First, split into temp training and test sets
X_temp, X_dev, y_temp, y_dev = train_test_split(X, y, test_size=0.05)

# Then, split the temp training set into final training and dev (validation) sets
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.05)

In [21]:
X_train.shape, X_test.shape, X_dev.shape

((10784, 200, 200, 3), (568, 200, 200, 3), (598, 200, 200, 3))

# Build and Train CNN Neural Network
Taking from pre-trained image CNN
Create base model from pre-trained model: MobileNet V2
Step 1: Create Base Model from MobileNet V2 model developed at Google.
Step 2: Freeze layers
Step 3: Train new layers on dataset
Step 4: Improve model via fine-tuning

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from keras.layers import Dense,GlobalAveragePooling2D

In [23]:
#create logs to view learning in real time
log_dir = os.path.join('Logs_MHI_ASL_mobile')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
#FD_SHAPE = (444, 444, 3)
MHI_SHAPE = (200,200,3)
#Try 6 labels instead of 2000 and increase (increments).
#reshaped_data = data.reshape(-1, 480, 640, 1)
# get the base model, exclude final dense layers - we will modify/output this
base_model = tf.keras.applications.MobileNetV2(input_shape = MHI_SHAPE,
                                               include_top = False,
                                               weights = 'imagenet',
                                               pooling='max')

In [27]:
actions.shape[0]

2000

In [26]:
model = tf.keras.Sequential()
model.add(base_model)
model.add(Dense(actions.shape[0], activation='softmax'))

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 100, 100, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 100, 100, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 100, 100, 32)         0         ['bn_Conv1[

 block_3_expand_relu (ReLU)  (None, 50, 50, 144)          0         ['block_3_expand_BN[0][0]']   
                                                                                                  
 block_3_pad (ZeroPadding2D  (None, 51, 51, 144)          0         ['block_3_expand_relu[0][0]'] 
 )                                                                                                
                                                                                                  
 block_3_depthwise (Depthwi  (None, 25, 25, 144)          1296      ['block_3_pad[0][0]']         
 seConv2D)                                                                                        
                                                                                                  
 block_3_depthwise_BN (Batc  (None, 25, 25, 144)          576       ['block_3_depthwise[0][0]']   
 hNormalization)                                                                                  
          

 block_6_depthwise_BN (Batc  (None, 13, 13, 192)          768       ['block_6_depthwise[0][0]']   
 hNormalization)                                                                                  
                                                                                                  
 block_6_depthwise_relu (Re  (None, 13, 13, 192)          0         ['block_6_depthwise_BN[0][0]']
 LU)                                                                                              
                                                                                                  
 block_6_project (Conv2D)    (None, 13, 13, 64)           12288     ['block_6_depthwise_relu[0][0]
                                                                    ']                            
                                                                                                  
 block_6_project_BN (BatchN  (None, 13, 13, 64)           256       ['block_6_project[0][0]']     
 ormalizat

                                                                                                  
 block_9_add (Add)           (None, 13, 13, 64)           0         ['block_8_add[0][0]',         
                                                                     'block_9_project_BN[0][0]']  
                                                                                                  
 block_10_expand (Conv2D)    (None, 13, 13, 384)          24576     ['block_9_add[0][0]']         
                                                                                                  
 block_10_expand_BN (BatchN  (None, 13, 13, 384)          1536      ['block_10_expand[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 block_10_expand_relu (ReLU  (None, 13, 13, 384)          0         ['block_10_expand_BN[0][0]']  
 )        

                                                                                                  
 block_13_expand_relu (ReLU  (None, 13, 13, 576)          0         ['block_13_expand_BN[0][0]']  
 )                                                                                                
                                                                                                  
 block_13_pad (ZeroPadding2  (None, 15, 15, 576)          0         ['block_13_expand_relu[0][0]']
 D)                                                                                               
                                                                                                  
 block_13_depthwise (Depthw  (None, 7, 7, 576)            5184      ['block_13_pad[0][0]']        
 iseConv2D)                                                                                       
                                                                                                  
 block_13_

 iseConv2D)                                                                                       
                                                                                                  
 block_16_depthwise_BN (Bat  (None, 7, 7, 960)            3840      ['block_16_depthwise[0][0]']  
 chNormalization)                                                                                 
                                                                                                  
 block_16_depthwise_relu (R  (None, 7, 7, 960)            0         ['block_16_depthwise_BN[0][0]'
 eLU)                                                               ]                             
                                                                                                  
 block_16_project (Conv2D)   (None, 7, 7, 320)            307200    ['block_16_depthwise_relu[0][0
                                                                    ]']                           
          

In [98]:
# Freeze the convolutional base
#base_model.trainable = False

In [99]:
#get the base model summary
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 100, 100, 32)         864       ['input_4[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 100, 100, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 100, 100, 32)         0         ['bn_Conv1[

 block_3_expand_relu (ReLU)  (None, 50, 50, 144)          0         ['block_3_expand_BN[0][0]']   
                                                                                                  
 block_3_pad (ZeroPadding2D  (None, 51, 51, 144)          0         ['block_3_expand_relu[0][0]'] 
 )                                                                                                
                                                                                                  
 block_3_depthwise (Depthwi  (None, 25, 25, 144)          1296      ['block_3_pad[0][0]']         
 seConv2D)                                                                                        
                                                                                                  
 block_3_depthwise_BN (Batc  (None, 25, 25, 144)          576       ['block_3_depthwise[0][0]']   
 hNormalization)                                                                                  
          

 block_6_depthwise_BN (Batc  (None, 13, 13, 192)          768       ['block_6_depthwise[0][0]']   
 hNormalization)                                                                                  
                                                                                                  
 block_6_depthwise_relu (Re  (None, 13, 13, 192)          0         ['block_6_depthwise_BN[0][0]']
 LU)                                                                                              
                                                                                                  
 block_6_project (Conv2D)    (None, 13, 13, 64)           12288     ['block_6_depthwise_relu[0][0]
                                                                    ']                            
                                                                                                  
 block_6_project_BN (BatchN  (None, 13, 13, 64)           256       ['block_6_project[0][0]']     
 ormalizat

                                                                                                  
 block_9_add (Add)           (None, 13, 13, 64)           0         ['block_8_add[0][0]',         
                                                                     'block_9_project_BN[0][0]']  
                                                                                                  
 block_10_expand (Conv2D)    (None, 13, 13, 384)          24576     ['block_9_add[0][0]']         
                                                                                                  
 block_10_expand_BN (BatchN  (None, 13, 13, 384)          1536      ['block_10_expand[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 block_10_expand_relu (ReLU  (None, 13, 13, 384)          0         ['block_10_expand_BN[0][0]']  
 )        

                                                                                                  
 block_13_expand_relu (ReLU  (None, 13, 13, 576)          0         ['block_13_expand_BN[0][0]']  
 )                                                                                                
                                                                                                  
 block_13_pad (ZeroPadding2  (None, 15, 15, 576)          0         ['block_13_expand_relu[0][0]']
 D)                                                                                               
                                                                                                  
 block_13_depthwise (Depthw  (None, 7, 7, 576)            5184      ['block_13_pad[0][0]']        
 iseConv2D)                                                                                       
                                                                                                  
 block_13_

 iseConv2D)                                                                                       
                                                                                                  
 block_16_depthwise_BN (Bat  (None, 7, 7, 960)            3840      ['block_16_depthwise[0][0]']  
 chNormalization)                                                                                 
                                                                                                  
 block_16_depthwise_relu (R  (None, 7, 7, 960)            0         ['block_16_depthwise_BN[0][0]'
 eLU)                                                               ]                             
                                                                                                  
 block_16_project (Conv2D)   (None, 7, 7, 320)            307200    ['block_16_depthwise_relu[0][0
                                                                    ]']                           
          

In [100]:
from tensorflow.keras.models import Model

# Add classification head/own layers
z = base_model.output
# If you want to include GlobalAveragePooling2D, you can directly use 'pooling=max' in the base model as done above
z = Dense(1024, activation='relu')(z)  # Dense layer 1
z = Dense(1024, activation='relu')(z)  # Dense layer 2
z = Dense(512, activation='relu')(z)   # Dense layer 3
preds = Dense(actions.shape[0], activation='softmax')(z)  # Final layer with softmax activation

# Create the full model
model = Model(inputs=base_model.input, outputs=preds)

In [101]:
#custom learning rate for Adam
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003) #0.001 can be changed.

In [102]:
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [103]:
#Train the model - MobileNetV2 as pretrained model.

model.fit(X_train, y_train, epochs=2000, validation_data = (X_dev, y_dev), batch_size = 32, callbacks=[tb_callback])


Epoch 1/2000
158/337 [=============>................] - ETA: 47s - loss: 7.6699 - categorical_accuracy: 1.9778e-04

KeyboardInterrupt: 

In [74]:
#save the model (for future load and training purposes)
model.save('MobileNetV2_first.keras')

### CNN without Pre-trained Model
Ignore this for now - this is a pure implementation without MobileNetV2 as the pre-trained model.

In [51]:

#without pre-train
model_FD = Sequential()
model_FD.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(96,10, 307200), strides=(2, 2), padding = 'same'))
model_FD.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer with 64 filters and kernel size of 5x5
model_FD.add(layers.Conv2D(filters=64, kernel_size=(5, 5), activation='relu', strides=(2, 2), padding = 'same'))
model_FD.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))

# Flattening the output of the previous layer to feed it into the dense layer
model_FD.add(layers.Flatten())

# First dense layer with 1024 neurons
model_FD.add(layers.Dense(units=1024, activation='relu'))

# Adding dropout to prevent overfitting
model_FD.add(layers.Dropout(rate=0.4))

#4 output neurons as current actions is 2.
# Output dense layer with 2 neurons (as per the number of classes) and softmax activation function
model_FD.add(layers.Dense(units=2, activation='softmax'))

ValueError: Exception encountered when calling layer "max_pooling2d_9" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_9/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,12,1,64].

Call arguments received by layer "max_pooling2d_9" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 12, 1, 64), dtype=float32)

In [12]:
#compile the model
model_FD.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_FD.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [13]:
#Generate Model summary to verify the architecture
model_FD.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 15, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 32)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 64)          51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 1, 1, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1024)              6

## VGG From Scratch
VGG16 is a CNN architecture. 

In [50]:
import keras,os
#from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
#from keras.preprocessing.image import ImageDataGenerator
#import numpy as np

In [51]:
MHI_SHAPE = (200,200,3)

model = Sequential()
#Block 1
model.add(Conv2D(input_shape=MHI_SHAPE,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Block 2
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Block 3
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Block 4
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Block 5
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Fully Connected Layers
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
#softmax units is dependent on the labels we want to extract out.
model.add(Dense(units=actions.shape[0], activation="softmax"))

#then, generate model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 200, 200, 64)      1792      
                                                                 
 conv2d_27 (Conv2D)          (None, 200, 200, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 100, 100, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_28 (Conv2D)          (None, 100, 100, 128)     73856     
                                                                 
 conv2d_29 (Conv2D)          (None, 100, 100, 128)     147584    
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 50, 50, 128)       0         
 ng2D)                                                

In [52]:
#custom learning rate for Adam

initial_learning_rate = 0.0003  # Starting learning rate
decay_steps = 100000           # After how many steps to apply decay
decay_rate = 0.96              # Decay rate
staircase = True               # Apply decay in a staircase fashion

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps,
    decay_rate,
    staircase=staircase
)

#adam_opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)  # Use the learning rate schedule here
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.0003) #0.001 can be changed.

In [58]:
model_VGG.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'], run_eagerly=True)

In [54]:
# Callbacks, implement model checkpoint and early stopping
from keras.callbacks import ModelCheckpoint, EarlyStopping
#val - represent validation metrics. therefore getting val_accuracy as metrics.
checkpoint = ModelCheckpoint("vgg16_best.h5", monitor='val_categorial_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
#early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')


#Tensorboard
#create logs to view learning in real time
log_path = os.path.join('Logs_MHI_ASL_VGG')
tb_callback = TensorBoard(log_dir=log_path)

my_callbacks = [
    #keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint("vgg16_best.h5", monitor='val_categorial_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto'),
    keras.callbacks.TensorBoard(log_dir=log_path),
]

In [26]:
#model.fit(X_train, y_train, epochs=2000, validation_data = (X_dev, y_dev), batch_size = 32, callbacks=my_callbacks)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [28]:
#create logs to view learning in real time using tensorboard
from keras.callbacks import CallbackList
#create logs to view learning in real time
log_dir = os.path.join('Logs_MHI_ASL_VGG')
tb_callback = TensorBoard(log_dir=log_dir)
callbacks_list = CallbackList([tensorboard_callback, checkpoint])

NameError: name 'tensorboard_callback' is not defined

In [59]:
# Finally, train the model using fit, callback for modelsavepoint and tensorboard (to visualize ML training)
model.fit(X_train, y_train, epochs=1000, validation_data=(X_dev, y_dev), batch_size=32)

Epoch 1/1000


TypeError: 'str' object is not callable

# Test the Values

In [ ]:
trained_cnn = load_model('MobileNetV2_first.keras')

In [ ]:
#Optional: continue to train the model (when increased the dataset)

In [75]:
res = model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [78]:
actions[np.argmax(res[0])]

'sorry'

In [79]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print(ytrue)
len(yhat)




1/1 [==============================] - 0s 275ms/step
[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1]


20

In [80]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 4],
        [4, 6]],

       [[6, 4],
        [4, 6]]], dtype=int64)

In [81]:
accuracy_score(ytrue, yhat)

0.6

In [61]:
!git status

fatal: not a git repository (or any of the parent directories): .git
